# Run a grid of models for comparison to cloudy (with fixed Temperature)

This version not only runs the models but also does everything that is needed to produce data for the resulsts. 

This should enable one to compare models for various model inputs

In [1]:
import os
os.getcwd()

'/Users/long/Projects/Python/release-models/cloudy/python'

In [2]:
import subprocess
import numpy as np
import shutil
from glob import glob
from astropy.io import ascii
from astropy.table import Table, join,vstack,hstack
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [3]:
VERSION='88a'

In [7]:
base='''System_type(star,cv,bh,agn,previous)                   bh

### Parameters for the Central Object
Central_object.mass(msol)                  0.8
Central_object.radius(cm)                  1e10
Binary.mass_sec(msol)                           0.4
Binary.period(hr)                             1000

### Parameters for the Disk (if there is one)
Disk.type(none,flat,vertically.extended,rmin>central.obj.rad)                 none

### Parameters for Boundary Layer or the compact object in an X-ray Binary or AGN
Central_object.radiation(yes,no)                  yes
Central_object.rad_type_to_make_wind(bb,models,power,cloudy,brems,mono)               cloudy
Central_object.luminosity(ergs/s)          %.3e
Central_object.power_law_index             -0.9
Central_object.geometry_for_source(sphere,lamp_post,bubble)               sphere
Central_object.cloudy.low_energy_break(ev)                .136
Central_object.cloudy.high_energy_break(ev)               20000

### Parameters describing the various winds or coronae in the system
Wind.number_of_components                  1
Wind.type(SV,star,hydro,corona,kwd,homologous,shell,imported)                shell

### Parameters associated with photon number, cycles,ionization and radiative transfer options
Photons_per_cycle                          10000000
Ionization_cycles                          30
Spectrum_cycles                            0
Wind.ionization(on.the.spot,ML93,LTE_tr,LTE_te,fixed,matrix_bb,matrix_pow,matrix_est)           matrix_pow
Line_transfer(pure_abs,pure_scat,sing_scat,escape_prob,thermal_trapping,macro_atoms_escape_prob,macro_atoms_thermal_trapping)     thermal_trapping
Wind.radiation(yes,no)                           no
Atomic_data                                data/standard80.dat
Surface.reflection.or.absorption(reflect,absorb,thermalized.rerad)              reflect
Wind_heating.extra_processes(none,adiabatic,nonthermal,both)                 none

### Parameters for Domain 0
Shell.wind_mdot(msol/yr)                   4.7e-20
Shell.wind.radmin(cm)                      1e11
Shell.wind.radmax(cm)                      1.00000000001e11
Shell.wind_v_at_rmin(cm)                   1
Shell.wind.v_at_rmax(cm)                   1.000010
Shell.wind.acceleration_exponent           1
Wind.t.init                                10000
Wind.filling_factor(1=smooth,<1=clumped)   1

### Parameters for Reverberation Modeling (if needed)
Reverb.type(none,photon,wind,matom)                 none

### Other parameters
Photon_sampling.approach(T_star,cv,yso,AGN,tde_bb,min_max_freq,user_bands,cloudy_test,wide,logarithmic)          cloudy_test
Photon_sampling.low_energy_limit(eV)            0.13
Photon_sampling.high_energy_limit(eV)                 1e8
'''

In [8]:
def run_one(log_ip=-8,version=VERSION):

    if log_ip<0:
        pfname='py%s_m%03.2f' % (version,-log_ip)
    else:
        pfname='py%s_p%03.2f' % (version,log_ip)
    

    f=open(pfname+'.pf','w')
    
    xlum=1e21 * 10**(log_ip+8.0)

    print(xlum)
    
    f.write(base % xlum)
    f.close()

    py='py'+version
  

    xcommand='mpirun -np 8 %s -p %s > %s.out.txt' % (py,pfname,pfname)
    print(xcommand)
    subprocess.run(xcommand,shell=True, check=True)

    try:
        signame= '%s.sig' % pfname
        f=open(signame)
    except:
        print('Error: Could not open %s ' % signame)
        return 1
        

    lines=f.readlines()
    last_line=lines[-1]
    print(last_line)
    if last_line.count('COMPLETE'):
        windsave2table = 'windsave2table%s' % version
        xcommand='%s %s >> %s.out.txt' % (windsave2table,pfname,pfname)
        # print(xcommand)
        subprocess.run(xcommand,shell=True, check=True)
        return 0
    return 1
              
run_one()    

1e+21
mpirun -np 8 py88a -p py88a_m8.00 > py88a_m8.00.out.txt
Sun Mar 31 14:10:23 2024     89.3 COMPLETE             py88a_m8.00



0

In [ ]:
def do_many(log_ipmin=-5,log_ipmax=8,npts=10,version=VERSION):
    log_ip=np.linspace(log_ipmin,log_ipmax,npts)
    # print(log_ip)
    i=1
    for one in log_ip:
        if one<0:
            pfname='py%s_m%03.2f' % (version,-one)
        else:
            pfname='py%s_p%03.2f' % (version,one)
        ival=run_one(one,version)
        if ival==0:
            os.remove(pfname+'.wind_save')
            shutil.rmtree('diag_%s' % pfname)
            files = glob('%s*spec*' % pfname)
            # print(files)
            for one_file in files:
                os.remove(one_file)
            print('Finished %s successfully (%d/%d)' % (pfname,i,len(log_ip)))
        else:
            print('Failed on %s  (%d/%d)' % (pfname,i,len(log_ip)))
        i+=1
                  
        
            
            
            

do_many(npts=100,version=VERSION)
    
    
    

# Now summarize the results

In [ ]:
xfiles=glob('py%s*master.txt'% VERSION)
# xfiles
xroots=[]
for one in xfiles:
    xroots.append(one.replace('.master.txt',''))

xroots=np.sort(xroots)

In [ ]:
def get_roots(prefix='py%s' % VERSION):
    xfiles=glob('%s*master.txt' % prefix)
    xorder=[]
    xroot=[]
    for one in xfiles:
        foo=one.replace('.master.txt','')
        xroot.append(foo)
        foo=foo.replace('%s_' % prefix,'')
        foo=foo.replace('p','+')
        foo=foo.replace('m','-')
        xorder.append(eval(foo))
    xtab=Table([xroot,xorder],names=['Root','IP'])
    xtab['IP'].format='.2f'
    xtab.sort('IP')
    return xtab
    

         
    
root_tab=get_roots()        

In [ ]:
root_tab

In [ ]:
def get_frac(root,element='H'):
    filename='%s.%s.frac.txt' % (root,element)
    try:
        xtab=ascii.read(filename)
        
    except:
        print('Error: could not read %s',filename)
        return None

    colnames=xtab.colnames
    # print(colnames)
    # i=3
    # while i<len(colnames):
    #    xtab.rename_column(colnames[i],colnames[i].replace('i',element))
    #    i+=1
    xtab.remove_column('r')
    xtab.remove_column('inwind')
    return xtab

get_frac('py%s_p0.25' % VERSION,'H')

    

In [ ]:
def make_summary_table(roots, element='H',version=VERSION):

    record=[]
    for one in roots:
        xtab=ascii.read(one+'.master.txt')
        ytab=xtab['i','converge','t_e','t_r','ip','xi']
        ztab=ytab[ytab['i']==1]
        # print(ztab)
        htab=get_frac(one,element)
        zztab=join(ztab,htab,join_type='left')
        # hetab=get_frac(one,'He')
        # zztab=join(zztab,hetab,join_type='left')
        # # print(zztab)
        record.append(zztab)

    xxx=vstack(record)

    

    xxx['ip'].format='.3e'
    xxx['xi'].format='.3e'
    xxx['t_e'].format='.3e'
    xxx['t_r'].format='.3e'
    

    xxx.write('Sum_py%s_%s_pl.txt' % (version,element),format='ascii.fixed_width_two_line',overwrite=True)
    return xxx



make_summary_table(root_tab['Root'])
        

In [ ]:
x=make_summary_table(root_tab['Root'],'He')
x=make_summary_table(root_tab['Root'],'C')
x=make_summary_table(root_tab['Root'],'N')
x=make_summary_table(root_tab['Root'],'O')
x=make_summary_table(root_tab['Root'],'Fe')
# x=make_summary_table(root_tab['Root'],'C')

In [ ]:
def do_all(version=VERSION):
    root_tab=get_roots('py%s' % version)
    sum_tab=make_summary_table(root_tab['Root'])
    print(root_tab.info)
    print(sum_tab.info)
    final=hstack([root_tab,sum_tab])
    final.info()
    return final

xfinal=do_all()

In [ ]:
plt.figure(1,(6,6))
plt.plot(xfinal['IP'],np.log10(xfinal['ip']),label='Culculated')
plt.plot([-8,8],[-8,8],':',label='Desired')
plt.xlabel('Desired IP')
plt.ylabel('Final ip')
plt.xlim(-8,8)
plt.ylim(-8,8)
plt.legend()

In [ ]:
xfinal.write('toast.txt',format='ascii.fixed_width_two_line',overwrite=True)